<b>Import dataset from AKUT and HA departments</b>

Help to import data
http://www.sthda.com/english/wiki/reading-data-from-excel-files-xls-xlsx-into-r


In [1]:
# install missing packages, should only run one time.
# install.packages("tidyverse")

In [2]:
# loading libraries
suppressWarnings(suppressMessages(  library(dplyr)  ))
suppressWarnings(suppressMessages(  library(tidyverse)))
suppressWarnings(suppressMessages(  library(lubridate)))
suppressWarnings(suppressMessages(  library(magrittr)))
suppressWarnings(suppressMessages(  library(stringr)))

In [3]:
# load data
rawdata <- read.table(file = "allergy.csv", sep = ";", header=TRUE)
pricelist <- read.table(file = "priser.csv", sep = ";", header=TRUE)

In [4]:
# check if the data is properly loaded.
rawdata %>%
head(3) %>%
print.data.frame()

  Beh.status Rekv.type Rekvirent                 Rekv.navn      Rkv.nr.
1     Afslut        PL     24600 alles Lægehus Vordingborg 104356128060
2     Afslut        PL     26085       Lægehuset I Benløse 104374153763
3     Afslut        PL     23981               Jørgen Rose 104385655277
       PMB.nr. Kvnt.nr.                     Kvnt.navn  Prvt.dato   Prvt.stedgeo
1 105991985581    57691 Inhalationsantigenpanel-IgE;P 11.05.2020 udveksling RSJ
2 104393874160    57691 Inhalationsantigenpanel-IgE;P 11.05.2020 udveksling RSJ
3 105945429248    57691 Inhalationsantigenpanel-IgE;P 11.05.2020 udveksling RSJ
  Prvt.kl Laboratorium   Res.producent Resultat Instrument Alder       DATO
1   12:35       SLAKBD SL,Slagelse KBA      0,1  SlaImcap2    69 11-05-2020
2   11:29       SLAKBD SL,Slagelse KBA    <0,10  SlaImcap1    23 11-05-2020
3   11:50       SLAKBD SL,Slagelse KBA      0,5  SlaImcap1    41 11-05-2020
    År Måned    Kvartal Uge Pris.øvrige LMV.pris Analysepris Pris.i.alt
1 2020   maj 2. 

In [5]:
# check datatypes
str(rawdata)

'data.frame':	49999 obs. of  25 variables:
 $ Beh.status   : Factor w/ 3 levels "","Afslut","Svar": 2 2 2 2 2 2 2 2 2 2 ...
 $ Rekv.type    : Factor w/ 8 levels "","AI","AM","PL",..: 4 4 4 4 4 4 4 4 4 4 ...
 $ Rekvirent    : Factor w/ 227 levels "","1411521","14834",..: 77 103 53 53 53 53 53 53 53 53 ...
 $ Rekv.navn    : Factor w/ 230 levels "","\235relæge Charlotte Stenby",..: 15 103 75 75 75 75 75 75 75 75 ...
 $ Rkv.nr.      : num  1.04e+11 1.04e+11 1.04e+11 1.04e+11 1.04e+11 ...
 $ PMB.nr.      : num  1.06e+11 1.04e+11 1.06e+11 1.06e+11 1.06e+11 ...
 $ Kvnt.nr.     : int  57691 57691 57691 13041 13098 13135 13080 13227 13167 13157 ...
 $ Kvnt.navn    : Factor w/ 9 levels "","Birk (t3)-IgE;P",..: 8 8 8 2 4 5 3 9 7 6 ...
 $ Prvt.dato    : Factor w/ 74 levels "","01.07.2020",..: 23 23 23 23 23 23 23 23 23 23 ...
 $ Prvt.stedgeo : Factor w/ 8 levels "","send til Webhotel",..: 8 8 8 8 8 8 8 8 8 8 ...
 $ Prvt.kl      : Factor w/ 506 levels "","06:01","06:44",..: 318 254 275 275 275 275 

In [6]:
# Which tests are the most frequent
rawdata %>%
group_by(Rekv.type) %>%
tally() %>%
arrange(desc(n))

Rekv.type,n
,38179
PL,9077
PS,1317
AM,823
AI,537
SE,34
UR,24
se,8


In [7]:
# cleaning dato and time type
#rawdata %>% unite(Dato.og.Tid, c("Prvt.dato", "Prvt.kl"))
#rawdata$Dato.og.Tid %<>% dmy_hm()
rawdata$DATO %<>% dmy()

# fix typo by character encoding
rawdata$Prvt.stedgeo %<>% str_replace_all("taget i Nåstved", "Taget i Næstved")

# cleaning casing of factors and recasting to factor type
rawdata$Rekv.type %<>% toupper() %>% as.factor()
rawdata$Rekvirent %<>% tolower() %>% as.factor()
rawdata$Prvt.stedgeo %<>% tolower() %>% as.factor()
rawdata$Res.producent %<>% tolower() %>% as.factor()
#rawdata$Kvartal %<>% tolower() %>% as.factor()

In [8]:
# de-select columns here
rawdata %<>% select(-Beh.status, -Laboratorium, -Res.producent, -Instrument)
str(rawdata)

'data.frame':	49999 obs. of  21 variables:
 $ Rekv.type   : Factor w/ 7 levels "","AI","AM","PL",..: 4 4 4 4 4 4 4 4 4 4 ...
 $ Rekvirent   : Factor w/ 226 levels "","1411521","14834",..: 77 103 53 53 53 53 53 53 53 53 ...
 $ Rekv.navn   : Factor w/ 230 levels "","\235relæge Charlotte Stenby",..: 15 103 75 75 75 75 75 75 75 75 ...
 $ Rkv.nr.     : num  1.04e+11 1.04e+11 1.04e+11 1.04e+11 1.04e+11 ...
 $ PMB.nr.     : num  1.06e+11 1.04e+11 1.06e+11 1.06e+11 1.06e+11 ...
 $ Kvnt.nr.    : int  57691 57691 57691 13041 13098 13135 13080 13227 13167 13157 ...
 $ Kvnt.navn   : Factor w/ 9 levels "","Birk (t3)-IgE;P",..: 8 8 8 2 4 5 3 9 7 6 ...
 $ Prvt.dato   : Factor w/ 74 levels "","01.07.2020",..: 23 23 23 23 23 23 23 23 23 23 ...
 $ Prvt.stedgeo: Factor w/ 8 levels "","send til webhotel",..: 8 8 8 8 8 8 8 8 8 8 ...
 $ Prvt.kl     : Factor w/ 506 levels "","06:01","06:44",..: 318 254 275 275 275 275 275 275 275 275 ...
 $ Resultat    : Factor w/ 271 levels "","<0,10",">100",..: 4 2 44 2 2 

In [9]:
# tidy up the pricelist
pricelist %<>%
select(Navn, BCC.nr., Pris)

# check if the data is properly loaded.
pricelist %>%
head(3) %>%
print.data.frame()

# check datatypes
str(pricelist)

                              Navn BCC.nr. Pris
1  17-Hydroxyprogesteron (0 min);P    4977  135
2 17-Hydroxyprogesteron (30 min);P    4978  135
3 17-Hydroxyprogesteron (60 min);P   21623  135
'data.frame':	1249 obs. of  3 variables:
 $ Navn   : Factor w/ 694 levels "","17-Hydroxyprogesteron (0 min);P",..: 2 3 4 5 6 7 8 10 11 14 ...
 $ BCC.nr.: int  4977 4978 21623 2460 1435 10267 2424 18242 18243 10504 ...
 $ Pris   : int  135 135 135 135 203 68 34 63 63 7 ...
